In [ ]:
import praw
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import re

# Reddit API setup
reddit = praw.Reddit(
    client_id="YOUR_REDDIT_CLIENT_ID",
    client_secret="YOUR_REDDIT_CLIENT_SECRET",
    user_agent="YOUR_USER_AGENT"
)

# Spotify API setup
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="YOUR_SPOTIFY_CLIENT_ID",
    client_secret="YOUR_SPOTIFY_CLIENT_SECRET",
    redirect_uri="YOUR_REDIRECT_URI",
    scope="playlist-modify-private"
))

def search_reddit_for_recommendations(subreddit_name, query, limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.search(query, limit=limit)
    
    recommendations = []
    for post in posts:
        post.comments.replace_more(limit=0)
        for comment in post.comments.list():
            recommendations.extend(extract_song_recommendations(comment.body))
    
    return recommendations

def extract_song_recommendations(text):
    # This is a simple extraction method. You might want to improve it for better accuracy.
    return re.findall(r'"([^"]*)" by ([^,\n]+)', text)

def create_spotify_playlist(name, tracks):
    user_id = sp.me()['id']
    playlist = sp.user_playlist_create(user_id, name, public=False)
    
    track_ids = []
    for track, artist in tracks:
        results = sp.search(q=f'track:{track} artist:{artist}', type='track')
        if results['tracks']['items']:
            track_ids.append(results['tracks']['items'][0]['id'])
    
    if track_ids:
        sp.playlist_add_items(playlist['id'], track_ids)
    
    return playlist['external_urls']['spotify']

def main():
    subreddit_name = "AskReddit"  # You can change this to any subreddit
    query = "song recommendations OR favorite songs"
    
    print("Searching Reddit for song recommendations...")
    recommendations = search_reddit_for_recommendations(subreddit_name, query)
    
    print("\nFound recommendations:")
    for track, artist in recommendations:
        print(f'"{track}" by {artist}')
    
    playlist_name = f"Reddit Song Recommendations from r/{subreddit_name}"
    print(f"\nCreating Spotify playlist: {playlist_name}")
    playlist_url = create_spotify_playlist(playlist_name, recommendations)
    
    print(f"\nPlaylist created! You can find it here: {playlist_url}")

if __name__ == "__main__":
    main()